# MCMC: A Julia application

## The Metropolis Hastings algorithm
The MH returns samples from the posterior distribution
$$
p(\theta|y) \propto p(y|\theta)p(\theta)
$$
Let $\bar{\theta}^{(s)}$, $s = 1,\ldots,S$ denote a sample from $p(\theta|y)$ obtained though application of the MH. With this sample we can _approximate_ various quantities that are of interest to the econometricians.

For instance, we may be interested in the posterior mean
$$
\mu_\theta  := \int \theta\, p(\theta|y) \, d\theta
$$
or the posterior variance
$$
\sigma^2_\theta := \int \left(\theta-\mu_\theta\right)^2p(\theta|y)\,d\theta
$$
Using $\bar{\theta}^{(s)}$ we can approximate $\mu_\theta$ and $\sigma^2_\theta$ by their sample counterparts
$$
\bar{\mu} = \frac{1}{S}\sum_{i=1}^S \bar{\theta}^{(s)},\quad \bar{\sigma}^2 = \frac{1}{S}\sum_{i=1}^S (\bar{\theta}^{(s)}-\bar{\mu})^2
$$
In general, we can approximate any moments of the posterior distribution
$$
\int h(\theta)p(\theta|y)\, d\theta
$$
by
$$
\frac{1}{S}\sum h(\bar{\theta}^{(s)}).
$$

To apply the MH we need to be able to _evaluate_ $p(y|\theta)p(\theta)$. _Evaluate_ means that we can get its value for different values of $\theta$.

## Example: the linear model with Student-t errors

To make things concrete, consider the following linear model
$$
y_t = x_t\beta + u_t, \,\, t = 1,\ldots,T.
$$
To accommodate outliers, the errors are assumed to be independently and identically generated by
$$
u_t \overset{iid}{\sim} t(0, \sigma, \nu).
$$
$$
f(u) = \Gamma\left(\frac{\nu+1}{2}\right)\left\{\Gamma\left(\frac{1}{2}\right)\Gamma\left(\frac{\nu}{2}\right)\sigma \right\}^{-1}\left[1+(u-\mu)^2/\nu\sigma^2\right]^{-(\nu+1)/2}
$$

The prior density is taken to be of the form
$$
p(\beta, \sigma) = p(\beta)p(\sigma),
$$
with improper marginal densities, i.e.
$$
p(\beta) \propto constant,\,p(\sigma) = \sigma^{-1}.
$$
Although $\nu$ is part of the specification of the model we will assume that $\nu$ is known.

Given the prior $p(\beta,\sigma) \propto \sigma^{-1}$ the posterior distribution has _kernel_
$$
p(\beta,\sigma|y,x) = \sigma^{-(T+1)}\prod_{t=1}^T \left[1 + (y_t - x_t\beta)^2/\nu\sigma^2 \right]^{-(\nu+1)/2} 
$$

## Generating data 

We generate fake data from the model above. To do this is useful to introduce the package `Distributions.jl`. This package makes working probability distributions very straightforward. 


In [ ]:
using Distributions
t = TDist(3)
println("The mean of `t` is ", mean(t), ", its variance is ", var(t)) 
## Generate a random sample from t
u = rand(t, 100);

If we want to generate a sample from a Student-t with mean $\mu$ and variance $\sigma^2$ we can do as follows

In [ ]:
mu = 5
sigma2 = 2
u = sqrt(sigma2)*rand(t,100)/sqrt(3) + mu;

The following function generate data from the linear model specified above taking as argument the size of the sample $T$, the number of regressor `k`, the degreees-of-freedom `nu`, the variance of the residual `sigma`, and the "true" value of the parametere vector `beta` which default to a zero vector of length `k`. (The regressors are generated from a standard normal distribution). 

In [ ]:
function simulate(T, k, nu, sigma, beta = zeros(k+1))
    # Generate "fake" linear model data
    t = TDist(nu)
    u = sigma*rand(t, T)/sqrt(nu)
    X = [ones(T,1) randn(T, k)]
    y =  X*beta + u
    y, X
end

To generate a draw of length $T=100$, with $\nu=3$ degrees-of-freedom from a model with three regressors

In [ ]:
y, X = simulate(100, 3, 3, 1);

## The frequentist way

The OLS estimates of the coefficient can be obtained by

In [ ]:
beta = X\y


An estimate of the variance of the coefficients is

In [ ]:
T, k = size(X)
uhat = y-X*beta
s2 = dot(uhat, uhat)/(length(uhat)-size(X,2))
varb = s2*inv(X'X)

The standard errors are

In [ ]:
stderr = diag(varb).^.5

Given the estimated coefficient and the standard errors, a 95% confidence intervals based on the asymptotic normal distribution the OLS estimator is easily obtained 

In [ ]:
[beta .- 1.96*stderr beta .+ 1.96*stderr]

An alternative way to obtain the OLS estimates is to use the `GLM.jl` package. `GLM.jl` is a very well designed package that makes it easy estimating and conducting inference in the context of Generalized Linear Models. The syntax through which a model is specified and estimated is very similar to that of `R`. 

In [ ]:
using GLM
using DataFrames
df = convert(DataFrame, X[:,2:end])
df[:y] = y
lm = lm(y~x1+x2+x3, df)

## Bayesian way
Now we turn to the Bayesian approach to estimation in the linear model with Student-t errors. Since inference is entirely based on the posterior distribution we start by writing a function the returns the log-posterior distribution of the model.  

`logposterior` takes 4 arguments. The parameter vector `theta`, the data `y` and `X`, and the degrees-of-freedom parameter. Notice that `theta` contains both the slopes of the linear model, i.e. $\beta_0$, $\beta_1$, $\ldots$,$\beta_k$ and the standard deviation of the residual parameter $\sigma$. As such, the dimension of `theta` is $k+1+1$.



In [ ]:
function logposterior(theta, y, X, nu)   
    T, k  = size(X)
    beta  = theta[1:k]
    sigma = theta[k+1]
    sigma <= 0 && return -Inf
    r = y-X*beta
    s = nu*sigma^2
    tmp = log(1+(r.^2)/s)
    loglik = -(T+1)*log(sigma) -((nu+1)/2)*sum(tmp)
    return loglik
end

The likelihood can be evaluated at different values of $\theta = (\beta_0,\beta_1,\ldots,\beta_k, \sigma)$:

In [ ]:
logposterior([0.1, 0.1, 0.2, 0.2, 0.8], y, X, 3)

In [ ]:
logposterior([0.4, 0.3, 0.23, 0.25, 0.84], y, X, 3)

It is important to realize that `logposterior` return `-Inf` when the last element of `theta` --- the standard deviation of the residual --- is equal or smaller than zero. This is necessary because the MH algorithm will invoke `logposterior` for values of `theta` generated by a proposal and there is not guarantee that these proposals have a value of $\sigma$ strictly larger than zero. 

In [ ]:
logposterior([0.4, 0.3, 0.23, 0.25, -0.84], y, X, 3)

### The MH algorithm
We now have almost all the pieces to run the MH algorithm to obtain draws from the posterior distribution. The algorithm is coded in the `metroprw` function. Arguments to this function are the number of draws `n_draw`, the initial value of the chain `theta`, the variance of the proposal distribution `Sigma`, the scalar tuning parameter `tau`, the data `y` and `X`, and the `nu` parameter. 

In [ ]:
function metroprw(n_draw, theta0, Sigma, tau, y, X, nu)
    T, k = size(X)
    NormDist = MvNormal(zeros(k+1), Sigma)
    ## Initialization the 
    theta      = zeros(k+1, n_draw+1);
    gamma_s    = logposterior(theta0, y, X, 3);
    theta[:,1] = theta0
    for s=1:n_draw
        theta_star =  theta[:,s] + tau*rand(NormDist, 1);
        gamma_star = logposterior(theta_star, y, X, 3);
        r = min(exp(gamma_star-gamma_s),1);
        u = rand()
        if(u<=r)
            theta[:, s+1] = theta_star
		          gamma_s = gamma_star
        else
            theta[:, s+1] = theta[:,s]
        end
     end

     return theta
end

To initialize the chain, we will follow the practice of starting from the maximum-a-posterior and using as variance of the proposal the inverse negative hessian of the logposterior evaluated at the maximum. To optimize the `logposterior` we use `Optim.jl` a package that containes different optimizers that are relatively easy to use. In what follows we uze the Broyden–Fletcher–Goldfarb–Shanno (`BFGS`) algorithm. We need to create a closure becuase `optim` expects a function in one argument. Also, it expects a that the function me minimized: since we instead want to maximize `logposterior`, we need to pass the negative logposterior. 


In [ ]:
using Optim
t0 = vec([beta' s2])
lp(theta) = -logposterior(theta, y, X, 3) 
res = optimize(lp, t0, BFGS())

In [ ]:
using Calculus
theta0 = Optim.minimizer(res)
Sigma = full(Hermitian(inv(Calculus.hessian(lp, theta0))))


In [ ]:

smpl = metroprw(1000, theta0, Sigma, 1.0, y, X, 3)

In [ ]:
using Plots;
gr();
Plots.histogram(smpl', layout=5)

In [ ]:
smpl = metroprw(100000, theta0, Sigma, 1.0, y, X, 3);

In [ ]:
Plots.histogram(smpl', layout=5)

To gauge whether the algorithm has converged, we can instect the **traceplot** which is a simple plot of the draws. 

In [ ]:
Plots.plot(smpl', layout=5)

Calculating posterior mean, posterior variance, posterior quantiles can be easily done by applying `mean`, `var`, and `quantile` to `smpl`.

In [ ]:
mean(smpl, 2)

In [ ]:
var(smpl,2)

In [ ]:
mapslices(quantile, smpl, 2)